In [54]:
from icecream import ic
from ordered_set import OrderedSet
from itertools import product
import itertools as _itertools
import stim
from idtcorev2 import jacobian_coefficient_calc
from pygsti.circuits.circuit import Circuit
from pygsti.baseobjs.label import Label
import math

# Update important parameters here!
VVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV

In [55]:
num_qubits = 2
max_weight = 2
term_dict = {("H", "XX"): 0.0001}

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
HS_index_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=1, max_weight=max_weight
)

pauli_node_attributes = list([p for p in HS_index_iterator])
ca_pauli_node_attributes = list(_itertools.combinations(pauli_node_attributes, 2))

def ca_pauli_weight_filter(pauli_pair, max_weight):
    used_indices_1 = set(
        i for i, ltr in enumerate(str(pauli_pair[0])[1:]) if ltr != "_"
    )
    used_indices_2 = set(
        i for i, ltr in enumerate(str(pauli_pair[1])[1:]) if ltr != "_"
    )
    union = used_indices_1.union(used_indices_2)
    if len(union) > 0 and len(union) <= max_weight:
        return True

ca_pauli_node_attributes = [
    ppair
    for ppair in ca_pauli_node_attributes
    if ca_pauli_weight_filter(ppair, max_weight)
]

measure_string_iterator = stim.PauliString.iter_all(
    num_qubits, min_weight=num_qubits
)
sign_iterator = list(product([1,-1], repeat=num_qubits))
prep_string_iterator = product([math.prod([item for item in sign_tuple]) for sign_tuple in sign_iterator],[p for p in measure_string_iterator])
measure_string_attributes = list([p for p in measure_string_iterator])
prep_string_attributes = list(a*b for a,b in prep_string_iterator)
prep_meas_pair = list(product(prep_string_attributes, measure_string_attributes))
awk_iterator = list(i[0] for i in product(sign_iterator, [p for p in measure_string_iterator]))

In [57]:
hs_error_gen_classes = "hs"
ca_error_gen_classes = "ca"


hs_experiment = list(
    product(
        hs_error_gen_classes,
        pauli_node_attributes,
        zip(awk_iterator, prep_string_attributes),
        measure_string_attributes,
    )
)
ca_experiment = list(
    product(
        ca_error_gen_classes,
        ca_pauli_node_attributes,
        zip(awk_iterator, prep_string_attributes),
        measure_string_attributes,
    )
)

In [ ]:
ca_pauli_node_attributes_pre_filter = list(_itertools.combinations(pauli_node_attributes, 2))
ca_pauli_node_attributes_pre_filter

[(stim.PauliString("+X_"), stim.PauliString("+Y_")),
 (stim.PauliString("+X_"), stim.PauliString("+Z_")),
 (stim.PauliString("+X_"), stim.PauliString("+_X")),
 (stim.PauliString("+X_"), stim.PauliString("+_Y")),
 (stim.PauliString("+X_"), stim.PauliString("+_Z")),
 (stim.PauliString("+X_"), stim.PauliString("+XX")),
 (stim.PauliString("+X_"), stim.PauliString("+XY")),
 (stim.PauliString("+X_"), stim.PauliString("+XZ")),
 (stim.PauliString("+X_"), stim.PauliString("+YX")),
 (stim.PauliString("+X_"), stim.PauliString("+YY")),
 (stim.PauliString("+X_"), stim.PauliString("+YZ")),
 (stim.PauliString("+X_"), stim.PauliString("+ZX")),
 (stim.PauliString("+X_"), stim.PauliString("+ZY")),
 (stim.PauliString("+X_"), stim.PauliString("+ZZ")),
 (stim.PauliString("+Y_"), stim.PauliString("+X_")),
 (stim.PauliString("+Y_"), stim.PauliString("+Z_")),
 (stim.PauliString("+Y_"), stim.PauliString("+_X")),
 (stim.PauliString("+Y_"), stim.PauliString("+_Y")),
 (stim.PauliString("+Y_"), stim.PauliString("+

In [59]:
ca_pauli_node_attributes

[(stim.PauliString("+X_"), stim.PauliString("+Y_")),
 (stim.PauliString("+X_"), stim.PauliString("+Z_")),
 (stim.PauliString("+X_"), stim.PauliString("+_X")),
 (stim.PauliString("+X_"), stim.PauliString("+_Y")),
 (stim.PauliString("+X_"), stim.PauliString("+_Z")),
 (stim.PauliString("+X_"), stim.PauliString("+XX")),
 (stim.PauliString("+X_"), stim.PauliString("+XY")),
 (stim.PauliString("+X_"), stim.PauliString("+XZ")),
 (stim.PauliString("+X_"), stim.PauliString("+YX")),
 (stim.PauliString("+X_"), stim.PauliString("+YY")),
 (stim.PauliString("+X_"), stim.PauliString("+YZ")),
 (stim.PauliString("+X_"), stim.PauliString("+ZX")),
 (stim.PauliString("+X_"), stim.PauliString("+ZY")),
 (stim.PauliString("+X_"), stim.PauliString("+ZZ")),
 (stim.PauliString("+Y_"), stim.PauliString("+X_")),
 (stim.PauliString("+Y_"), stim.PauliString("+Z_")),
 (stim.PauliString("+Y_"), stim.PauliString("+_X")),
 (stim.PauliString("+Y_"), stim.PauliString("+_Y")),
 (stim.PauliString("+Y_"), stim.PauliString("+

In [60]:
import pandas as pd

# df = pd.DataFrame()
jacobian_coef_dict = {"index": OrderedSet(), "columns": OrderedSet()}
data = {}

# These come back as class, index, prep_str, meas_str, observ_str: coef
# I THINK this is right, should double check and write unit tests
for key in hs_experiment + ca_experiment:
    elt = jacobian_coefficient_calc(*key)
    for el in elt:
        if el:
            observable = ",".join(str(s)[:] for s in el[-5:-1])
            sign_string = el[-4]
            egen = ",".join(str(s) for s in el[:-5])
            coef = int(el[-1])
            # ic(observable)
            jacobian_coef_dict["index"].add(observable)
            jacobian_coef_dict["columns"].add(egen)
            if data.get(egen):
                data[egen].append(coef)
            else:
                data[egen] = [coef]

df = pd.DataFrame(data, index= jacobian_coef_dict["index"], columns=jacobian_coef_dict["columns"])

In [61]:
jacobian_coef_dict["columns"]
# jacobian_coef_dict["index"]

OrderedSet(['H,+X_', 'H,+Y_', 'H,+Z_', 'H,+_X', 'H,+_Y', 'H,+_Z', 'H,+XX', 'H,+XY', 'H,+XZ', 'H,+YX', 'H,+YY', 'H,+YZ', 'H,+ZX', 'H,+ZY', 'H,+ZZ', 'S,+X_', 'S,+Y_', 'S,+Z_', 'S,+_X', 'S,+_Y', 'S,+_Z', 'S,+XX', 'S,+XY', 'S,+XZ', 'S,+YX', 'S,+YY', 'S,+YZ', 'S,+ZX', 'S,+ZY', 'S,+ZZ', 'C,+X_,+Y_', 'C,+X_,+Z_', 'C,+X_,+_X', 'C,+X_,+_Y', 'C,+X_,+_Z', 'C,+X_,+XX', 'C,+X_,+XY', 'C,+X_,+XZ', 'C,+X_,+YX', 'C,+X_,+YY', 'C,+X_,+YZ', 'C,+X_,+ZX', 'C,+X_,+ZY', 'C,+X_,+ZZ', 'C,+Y_,+X_', 'C,+Y_,+Z_', 'C,+Y_,+_X', 'C,+Y_,+_Y', 'C,+Y_,+_Z', 'C,+Y_,+XX', 'C,+Y_,+XY', 'C,+Y_,+XZ', 'C,+Y_,+YX', 'C,+Y_,+YY', 'C,+Y_,+YZ', 'C,+Y_,+ZX', 'C,+Y_,+ZY', 'C,+Y_,+ZZ', 'C,+Z_,+X_', 'C,+Z_,+Y_', 'C,+Z_,+_X', 'C,+Z_,+_Y', 'C,+Z_,+_Z', 'C,+Z_,+XX', 'C,+Z_,+XY', 'C,+Z_,+XZ', 'C,+Z_,+YX', 'C,+Z_,+YY', 'C,+Z_,+YZ', 'C,+Z_,+ZX', 'C,+Z_,+ZY', 'C,+Z_,+ZZ', 'C,+_X,+X_', 'C,+_X,+Y_', 'C,+_X,+Z_', 'C,+_X,+_Y', 'C,+_X,+_Z', 'C,+_X,+XX', 'C,+_X,+XY', 'C,+_X,+XZ', 'C,+_X,+YX', 'C,+_X,+YY', 'C,+_X,+YZ', 'C,+_X,+ZX', 'C,+_X,+ZY', 'C,

In [62]:
df.to_csv("ick.csv")

In [63]:
import numpy as np
np.set_printoptions(precision=1, linewidth=1000)

In [64]:
from pygsti.io import read_results_from_dir, load_model
from pygsti.models import ExplicitOpModel
# read_results_from_dir("/Users/jkskolf/2Q-GST/2024_11_26_torino_2q_gst_cleaned_protocoldata_results")
exp_op_model = ExplicitOpModel.read("C://Users/jkskolf/Downloads/2024_11_26_torino_2q_gst_cleaned_protocoldata_results/2024_11_26_torino_2q_gst_cleaned_protocoldata_results/results/StandardGST/estimates_CPTPLND/models_stdgaugeopt.json")

In [65]:
print(exp_op_model)

rho0 = TPState with dimension 16
 0.50   0 0.03 0.48   0   0   0   0 0.03   0   0 0.03 0.50   0 0.03 0.48


Mdefault = TPPOVM with effect vectors:
00: FullPOVMEffect with dimension 16
 0.49   0-0.02 0.47   0   0   0   0-0.01   0   0-0.01 0.48   0-0.02 0.46

01: FullPOVMEffect with dimension 16
 0.51   0 0.02-0.47   0   0   0   0-0.02   0   0 0.01 0.50   0 0.02-0.46

10: FullPOVMEffect with dimension 16
 0.49   0-0.02 0.46   0   0   0   0 0.01   0   0 0.01-0.49   0 0.02-0.46

11: ComplementPOVMEffect with dimension 16
 0.51   0 0.02-0.46   0   0   0   0 0.01   0   0-0.02-0.50   0-0.02 0.46



Gxpi2:Q87 = 
FullTPOp with shape (16, 16)
 1.00   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 1.00   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0 1.00   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0 1.00   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0 1.00   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0 1.00   0

In [66]:
exp_op_model.operations[("Gi","Q87","Q88")] = exp_op_model.operations[Label(())].copy()
exp_op_model._rebuild_paramvec()

## Pull error generator params from model

In [67]:
from pygsti.report.reportables import log_gti_and_projections
from pygsti.baseobjs import BuiltinBasis
some_sort_of_thing = log_gti_and_projections(exp_op_model.operations[("Gi","Q87","Q88")].to_dense(), np.eye(4**num_qubits), BuiltinBasis("pp", 16))

In [68]:
print(some_sort_of_thing['H projections'])

[ 7.8e-05  7.6e-05 -9.0e-05  1.4e-04  1.7e-05 -1.0e-05 -3.9e-05  9.7e-05 -3.1e-05  5.2e-05 -1.7e-05 -1.2e-04  4.1e-05 -5.5e-05  3.5e-05]


In [69]:
import pygsti
from pygsti.extras import idletomography as idt
from pygsti.baseobjs import Label
gates = ["Gi", "Gxpi2", "Gzpi2", "Gcphase"]
max_lengths = [1, 2, 4, 8, 16, 32, 64, 128]
pspec = pygsti.processors.QubitProcessorSpec(
        num_qubits, gates, geometry="line", nonstd_gate_unitaries={(): num_qubits, "Gi": np.eye((2**num_qubits))},
        availability={"Gi": [tuple(i for i in ["Q87","Q88"])]}, qubit_labels=["Q87","Q88"]
    )
mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
global_idle_string = [Label("Gi", tuple(i for i in ["Q87","Q88"]))]
idle_experiments = idt.make_idle_tomography_list(
        num_qubits, max_lengths, paulidicts, idle_string=global_idle_string, qubit_labels=["Q87","Q88"]
    )
idle_experiments

ic| fidpairs: [(State[+X+X], State[+X+X]),
               (State[+X+X], State[+X+Y]),
               (State[+X+X], State[+X+Z]),
               (State[+X+X], State[+Y+X]),
               (State[+X+X], State[+Y+Y]),
               (State[+X+X], State[+Y+Z]),
               (State[+X+X], State[+Z+X]),
               (State[+X+X], State[+Z+Y]),
               (State[+X+X], State[+Z+Z]),
               (State[+X+Y], State[+X+X]),
               (State[+X+Y], State[+X+Y]),
               (State[+X+Y], State[+X+Z]),
               (State[+X+Y], State[+Y+X]),
               (State[+X+Y], State[+Y+Y]),
               (State[+X+Y], State[+Y+Z]),
               (State[+X+Y], State[+Z+X]),
               (State[+X+Y], State[+Z+Y]),
               (State[+X+Y], State[+Z+Z]),
               (State[+X+Z], State[+X+X]),
               (State[+X+Z], State[+X+Y]),
               (State[+X+Z], State[+X+Z]),
               (State[+X+Z], State[+Y+X]),
               (State[+X+Z], State[+Y+Y]),
           

[Circuit(Gxpi2:Q87Gzpi2:Q87Gi:Q87:Q88Gxpi2:Q87Gzpi2:Q87Gxpi2:Q87Gzpi2:Q87Gxpi2:Q87Gzpi2:Q87@(Q87,Q88)),
 Circuit(Gxpi2:Q87Gzpi2:Q87Gi:Q87:Q88Gi:Q87:Q88Gxpi2:Q87Gzpi2:Q87Gxpi2:Q87Gzpi2:Q87Gxpi2:Q87Gzpi2:Q87@(Q87,Q88)),
 Circuit(Gxpi2:Q87Gzpi2:Q87Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gxpi2:Q87Gzpi2:Q87Gxpi2:Q87Gzpi2:Q87Gxpi2:Q87Gzpi2:Q87@(Q87,Q88)),
 Circuit(Gxpi2:Q87Gzpi2:Q87Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gxpi2:Q87Gzpi2:Q87Gxpi2:Q87Gzpi2:Q87Gxpi2:Q87Gzpi2:Q87@(Q87,Q88)),
 Circuit(Gxpi2:Q87Gzpi2:Q87Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gxpi2:Q87Gzpi2:Q87Gxpi2:Q87Gzpi2:Q87Gxpi2:Q87Gzpi2:Q87@(Q87,Q88)),
 Circuit(Gxpi2:Q87Gzpi2:Q87Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:Q87:Q88Gi:

In [70]:
print(paulidicts)

({'X': ('Gxpi2', 'Gzpi2'), 'Y': ('Gxpi2', 'Gxpi2', 'Gxpi2'), 'Z': (), '-X': ('Gxpi2', 'Gzpi2', 'Gxpi2', 'Gzpi2', 'Gxpi2', 'Gzpi2'), '-Y': ('Gxpi2',), '-Z': ('Gxpi2', 'Gxpi2')}, {'X': ('Gxpi2', 'Gzpi2', 'Gxpi2', 'Gzpi2', 'Gxpi2', 'Gzpi2'), 'Y': ('Gxpi2',), 'Z': (), '-X': ('Gxpi2', 'Gzpi2'), '-Y': ('Gxpi2', 'Gxpi2', 'Gxpi2'), '-Z': ('Gxpi2', 'Gxpi2')})


In [71]:
# mdl_datagen = pygsti.models.create_crosstalk_free_model(
# pspec, lindblad_error_coeffs={"Gi": term_dict},lindblad_parameterization="GLND")

# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(
    exp_op_model, idle_experiments, 10000000, seed=8675309, sample_error="none"
)


In [72]:
from idttools import allerrors, all_full_length_observables, alloutcomes
import collections as _collections
from pygsti.circuits.circuit import Circuit as _Circuit
def report_observed_rates(nqubits,
    dataset,
    max_lengths,
    pauli_basis_dicts,
    maxweight=2,
    idle_string=global_idle_string,
    qubit_labels = None):
    
    all_fidpairs = dict(enumerate(idt.idle_tomography_fidpairs(nqubits)))
    # ic(all_fidpairs)
    if nqubits == 1:  # special case where line-labels may be ('*',)
        if len(dataset) > 0:
            first_circuit = list(dataset.keys())[0]
            line_labels = first_circuit.line_labels
        else:
            line_labels = (0,)
        GiStr = _Circuit(idle_string, line_labels=line_labels)
    else:
        GiStr = _Circuit(idle_string, num_lines=nqubits)
    obs_infos = dict()
    errors = allerrors(nqubits, maxweight)
    fit_order = 1
    observed_error_rates = {}
    obs_error_rates_by_exp = {}
    whatever = {}
    for ifp, pauli_fidpair in all_fidpairs.items():
        all_outcomes = idt.idttools.allobservables(pauli_fidpair[1], maxweight)
        infos_for_this_fidpair = _collections.OrderedDict()
        # ic(pauli_fidpair)
        for j, out in enumerate(all_outcomes):
            info = idt.compute_observed_err_rate(
                dataset,
                pauli_fidpair,
                pauli_basis_dicts,
                GiStr,
                out,
                max_lengths,
                fit_order,
                qubit_labels=qubit_labels
            )

            infos_for_this_fidpair[out] = info
            
            obs_infos[ifp] = infos_for_this_fidpair
            observed_error_rates[ifp] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ]
            obs_error_rates_by_exp[str(pauli_fidpair[0]).replace("+",""), str(pauli_fidpair[1]).replace("+",""), str(out).replace("I","_")] = [
                info["rate"] for info in infos_for_this_fidpair.values()
            ][-1]
        whatever[pauli_fidpair] = 1
    return observed_error_rates, obs_error_rates_by_exp

In [73]:
observed_error_rates, obs_error_rates_by_exp = report_observed_rates(num_qubits, ds, max_lengths, paulidicts, qubit_labels=["Q87","Q88"])

ic| fidpairs: [(State[+X+X], State[+X+X]),
               (State[+X+X], State[+X+Y]),
               (State[+X+X], State[+X+Z]),
               (State[+X+X], State[+Y+X]),
               (State[+X+X], State[+Y+Y]),
               (State[+X+X], State[+Y+Z]),
               (State[+X+X], State[+Z+X]),
               (State[+X+X], State[+Z+Y]),
               (State[+X+X], State[+Z+Z]),
               (State[+X+Y], State[+X+X]),
               (State[+X+Y], State[+X+Y]),
               (State[+X+Y], State[+X+Z]),
               (State[+X+Y], State[+Y+X]),
               (State[+X+Y], State[+Y+Y]),
               (State[+X+Y], State[+Y+Z]),
               (State[+X+Y], State[+Z+X]),
               (State[+X+Y], State[+Z+Y]),
               (State[+X+Y], State[+Z+Z]),
               (State[+X+Z], State[+X+X]),
               (State[+X+Z], State[+X+Y]),
               (State[+X+Z], State[+X+Z]),
               (State[+X+Z], State[+Y+X]),
               (State[+X+Z], State[+Y+Y]),
           

In [74]:
obs_rats = [v for v in obs_error_rates_by_exp.values()]

In [75]:
df.index

Index(['+XX,(1, 1),+XX,+X_', '+XX,(1, 1),+XX,+_X', '+XX,(1, 1),+XX,+XX',
       '+XX,(1, 1),+XY,+X_', '+XX,(1, 1),+XY,+_Y', '+XX,(1, 1),+XY,+XY',
       '+XX,(1, 1),+XZ,+X_', '+XX,(1, 1),+XZ,+_Z', '+XX,(1, 1),+XZ,+XZ',
       '+XX,(1, 1),+YX,+Y_',
       ...
       '+ZZ,(-1, -1),+YZ,+YZ', '+ZZ,(-1, -1),+ZX,+Z_', '+ZZ,(-1, -1),+ZX,+_X',
       '+ZZ,(-1, -1),+ZX,+ZX', '+ZZ,(-1, -1),+ZY,+Z_', '+ZZ,(-1, -1),+ZY,+_Y',
       '+ZZ,(-1, -1),+ZY,+ZY', '+ZZ,(-1, -1),+ZZ,+Z_', '+ZZ,(-1, -1),+ZZ,+_Z',
       '+ZZ,(-1, -1),+ZZ,+ZZ'],
      dtype='object', length=972)

In [76]:
obs_error_rates_by_exp

{('XX', 'XX', ' X_'): -0.00014530716874633956,
 ('XX', 'XX', ' _X'): -0.00021426066166313227,
 ('XX', 'XX', ' XX'): -0.00010989649414019611,
 ('XX', 'XY', ' X_'): -0.00014530716874633956,
 ('XX', 'XY', ' _Y'): -0.00021426066166313227,
 ('XX', 'XY', ' XY'): -0.00010989649414019611,
 ('XX', 'XZ', ' X_'): -0.00014530716874633956,
 ('XX', 'XZ', ' _Z'): -0.00021426066166313227,
 ('XX', 'XZ', ' XZ'): -0.00010989649414019611,
 ('XX', 'YX', ' Y_'): -0.00016025385688525168,
 ('XX', 'YX', ' _X'): -0.00021034000409000597,
 ('XX', 'YX', ' YX'): -9.80438762442349e-05,
 ('XX', 'YY', ' Y_'): -0.00016025385688525168,
 ('XX', 'YY', ' _Y'): -0.00021034000409000597,
 ('XX', 'YY', ' YY'): -9.80438762442349e-05,
 ('XX', 'YZ', ' Y_'): -0.00016025385688525168,
 ('XX', 'YZ', ' _Z'): -0.00021034000409000597,
 ('XX', 'YZ', ' YZ'): -9.80438762442349e-05,
 ('XX', 'ZX', ' Z_'): -0.0001446923062629724,
 ('XX', 'ZX', ' _X'): -0.000212079971783966,
 ('XX', 'ZX', ' ZX'): -0.00010753585820649067,
 ('XX', 'ZY', ' Z_'): 

In [105]:
for col in df.columns:
    print(col)

H,+X_
H,+Y_
H,+Z_
H,+_X
H,+_Y
H,+_Z
H,+XX
H,+XY
H,+XZ
H,+YX
H,+YY
H,+YZ
H,+ZX
H,+ZY
H,+ZZ
S,+X_
S,+Y_
S,+Z_
S,+_X
S,+_Y
S,+_Z
S,+XX
S,+XY
S,+XZ
S,+YX
S,+YY
S,+YZ
S,+ZX
S,+ZY
S,+ZZ
C,+X_,+Y_
C,+X_,+Z_
C,+X_,+_X
C,+X_,+_Y
C,+X_,+_Z
C,+X_,+XX
C,+X_,+XY
C,+X_,+XZ
C,+X_,+YX
C,+X_,+YY
C,+X_,+YZ
C,+X_,+ZX
C,+X_,+ZY
C,+X_,+ZZ
C,+Y_,+X_
C,+Y_,+Z_
C,+Y_,+_X
C,+Y_,+_Y
C,+Y_,+_Z
C,+Y_,+XX
C,+Y_,+XY
C,+Y_,+XZ
C,+Y_,+YX
C,+Y_,+YY
C,+Y_,+YZ
C,+Y_,+ZX
C,+Y_,+ZY
C,+Y_,+ZZ
C,+Z_,+X_
C,+Z_,+Y_
C,+Z_,+_X
C,+Z_,+_Y
C,+Z_,+_Z
C,+Z_,+XX
C,+Z_,+XY
C,+Z_,+XZ
C,+Z_,+YX
C,+Z_,+YY
C,+Z_,+YZ
C,+Z_,+ZX
C,+Z_,+ZY
C,+Z_,+ZZ
C,+_X,+X_
C,+_X,+Y_
C,+_X,+Z_
C,+_X,+_Y
C,+_X,+_Z
C,+_X,+XX
C,+_X,+XY
C,+_X,+XZ
C,+_X,+YX
C,+_X,+YY
C,+_X,+YZ
C,+_X,+ZX
C,+_X,+ZY
C,+_X,+ZZ
C,+_Y,+X_
C,+_Y,+Y_
C,+_Y,+Z_
C,+_Y,+_X
C,+_Y,+_Z
C,+_Y,+XX
C,+_Y,+XY
C,+_Y,+XZ
C,+_Y,+YX
C,+_Y,+YY
C,+_Y,+YZ
C,+_Y,+ZX
C,+_Y,+ZY
C,+_Y,+ZZ
C,+_Z,+X_
C,+_Z,+Y_
C,+_Z,+Z_
C,+_Z,+_X
C,+_Z,+_Y
C,+_Z,+XX
C,+_Z,+XY
C,+_Z,+XZ
C,+_Z,+YX
C,+_Z,+YY
C,+_Z,+YZ
C,+_Z,+ZX


In [77]:
j = df.to_numpy()

In [78]:
j.shape

(972, 450)

In [79]:
np.linalg.matrix_rank(j)

240

In [80]:
jinv = np.linalg.pinv(j)

In [81]:
intrins_errs = jinv @ obs_rats

# Supposed intrinsic error rates (wrong)

In [82]:
x = dict(zip(df.columns,intrins_errs))

In [83]:
len(x.keys())

450

In [84]:
len(df.columns)

450

In [85]:
len(df.index)

972

In [86]:
len(obs_error_rates_by_exp)

972

In [87]:
len(idle_experiments)

2592

In [88]:
len(ca_pauli_node_attributes)

210

In [89]:
len(hs_experiment)

9720

In [90]:
len(ca_experiment)

136080

In [91]:
450*972

437400

In [92]:
for i in hs_experiment:
    if i[0] == 'h':
       print(i)

('h', stim.PauliString("+X_"), ((1, 1), stim.PauliString("+XX")), stim.PauliString("+XX"))
('h', stim.PauliString("+X_"), ((1, 1), stim.PauliString("+XX")), stim.PauliString("+XY"))
('h', stim.PauliString("+X_"), ((1, 1), stim.PauliString("+XX")), stim.PauliString("+XZ"))
('h', stim.PauliString("+X_"), ((1, 1), stim.PauliString("+XX")), stim.PauliString("+YX"))
('h', stim.PauliString("+X_"), ((1, 1), stim.PauliString("+XX")), stim.PauliString("+YY"))
('h', stim.PauliString("+X_"), ((1, 1), stim.PauliString("+XX")), stim.PauliString("+YZ"))
('h', stim.PauliString("+X_"), ((1, 1), stim.PauliString("+XX")), stim.PauliString("+ZX"))
('h', stim.PauliString("+X_"), ((1, 1), stim.PauliString("+XX")), stim.PauliString("+ZY"))
('h', stim.PauliString("+X_"), ((1, 1), stim.PauliString("+XX")), stim.PauliString("+ZZ"))
('h', stim.PauliString("+X_"), ((1, 1), stim.PauliString("+XY")), stim.PauliString("+XX"))
('h', stim.PauliString("+X_"), ((1, 1), stim.PauliString("+XY")), stim.PauliString("+XY"))

In [93]:
240*972

233280

In [94]:
len(df.index)

972

In [95]:
30*972

29160

In [96]:
def partially_fill_dataframe(df, values):
    """
    Fills a DataFrame with values from a list, left to right. 
    Stops when the list is exhausted or all NaN values are filled.
    """

    flattened_df = df.values.flatten()
    for i in range(len(flattened_df)):
        print(i)
        print(flattened_df[i])
        if True:
            if values:
                flattened_df[i] = str(values.pop(0))
            else:
                break

    return pd.DataFrame(flattened_df.reshape(df.shape), columns=df.columns)

In [97]:
df = df.astype('string')
ohno = partially_fill_dataframe(df, hs_experiment)

0
0
1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
0
12
0
13
0
14
0
15
0
16
-2
17
-2
18
0
19
0
20
0
21
0
22
0
23
0
24
-2
25
-2
26
-2
27
-2
28
-2
29
-2
30
0
31
0
32
0
33
0
34
0
35
0
36
0
37
0
38
0
39
0
40
0
41
0
42
0
43
0
44
0
45
0
46
0
47
0
48
0
49
0
50
0
51
0
52
-4
53
0
54
0
55
0
56
0
57
0
58
0
59
0
60
0
61
0
62
0
63
0
64
0
65
0
66
0
67
0
68
0
69
-4
70
0
71
0
72
0
73
0
74
0
75
0
76
0
77
0
78
0
79
0
80
0
81
0
82
0
83
0
84
0
85
0
86
0
87
0
88
0
89
0
90
0
91
0
92
0
93
0
94
0
95
0
96
0
97
0
98
0
99
0
100
0
101
0
102
0
103
0
104
0
105
0
106
0
107
0
108
0
109
0
110
0
111
0
112
0
113
0
114
0
115
0
116
0
117
0
118
0
119
0
120
0
121
0
122
0
123
0
124
0
125
0
126
0
127
0
128
0
129
0
130
0
131
0
132
0
133
0
134
0
135
0
136
0
137
0
138
0
139
0
140
0
141
0
142
0
143
0
144
0
145
0
146
0
147
0
148
0
149
0
150
0
151
0
152
0
153
0
154
0
155
0
156
0
157
-4
158
0
159
0
160
0
161
0
162
0
163
0
164
0
165
0
166
0
167
0
168
0
169
0
170
0
171
0
172
0
173
0
174
0
175
0
176
0
177
0
178
0
179
0
180
0
181
0
182
0
1

In [98]:
ohno.to_csv("ohNO.csv")

In [99]:
hs_experiment

[]

In [100]:
df

H,+X_ H,+Y_ H,+Z_ H,+_X H,+_Y H,+_Z H,+XX H,+XY H,+XZ  \
+XX,(1, 1),+XX,+X_       0     0     0     0     0     0     0     0     0   
+XX,(1, 1),+XX,+_X       0     0     0     0     0     0     0     0     0   
+XX,(1, 1),+XX,+XX       0     0     0     0     0     0     0     0     0   
+XX,(1, 1),+XY,+X_       0     0     0     0     0     0     0     0     0   
+XX,(1, 1),+XY,+_Y       0     0     0     0     0     2     0     0     2   
...                    ...   ...   ...   ...   ...   ...   ...   ...   ...   
+ZZ,(-1, -1),+ZY,+_Y     0     0     0     2     0     0     0     0     0   
+ZZ,(-1, -1),+ZY,+ZY     0     0     0    -2     0     0     0     0     0   
+ZZ,(-1, -1),+ZZ,+Z_     0     0     0     0     0     0     0     0     0   
+ZZ,(-1, -1),+ZZ,+_Z     0     0     0     0     0     0     0     0     0   
+ZZ,(-1, -1),+ZZ,+ZZ     0     0     0     0     0     0     0     0     0   

                     H,+YX  ... A,+ZZ,+_Y A,+ZZ,+_Z A,+ZZ,+XX A,+ZZ,+XY  \
+XX,(1, 1),+XX,+X_       0  ...         0         0         0         0   
+XX,(1, 1),+XX,+_X       0  ...         0         0         0         0   
+XX,(1, 1),+XX,+XX       0  ...         0         0         0         0   
+XX,(1, 1),+XY,+X_       0  ...         0         0         0         0   
+XX,(1, 1),+XY,+_Y       0  ...         0         0         0         0   
...                    ...  ...       ...       ...       ...       ...   
+ZZ,(-1, -1),+ZY,+_Y     0  ...         0         0         0         0   
+ZZ,(-1, -1),+ZY,+ZY     0  ...         0         0         0         0   
+ZZ,(-1, -1),+ZZ,+Z_     0  ...         0         0         0         0   
+ZZ,(-1, -1),+ZZ,+_Z     0  ...         0         0         0         0   
+ZZ,(-1, -1),+ZZ,+ZZ     0  ...         0         0         0         0   

                     A,+ZZ,+XZ A,+ZZ,+YX A,+ZZ,+YY A,+ZZ,+YZ A,+ZZ,+ZX  \
+XX,(1, 1),+XX,+X_           0         0         0         4         0   
+XX,(1, 1),+XX,+_X           0         0         0         4         0   
+XX,(1, 1),+XX,+XX           0         0         0         0         0   
+XX,(1, 1),+XY,+X_           0         0         0         4         0   
+XX,(1, 1),+XY,+_Y           0         0         0         0        -4   
...                        ...       ...       ...       ...       ...   
+ZZ,(-1, -1),+ZY,+_Y         0         0         0         0        -4   
+ZZ,(-1, -1),+ZY,+ZY         0         0         0         0         4   
+ZZ,(-1, -1),+ZZ,+Z_         0         0         0         0         0   
+ZZ,(-1, -1),+ZZ,+_Z         0         0         0         0         0   
+ZZ,(-1, -1),+ZZ,+ZZ         0         0         0         0         0   

                     A,+ZZ,+ZY  
+XX,(1, 1),+XX,+X_           4  
+XX,(1, 1),+XX,+_X           4  
+XX,(1, 1),+XX,+XX           0  
+XX,(1, 1),+XY,+X_           4  
+XX,(1, 1),+XY,+_Y           0  
...                        ...  
+ZZ,(-1, -1),+ZY,+_Y         0  
+ZZ,(-1, -1),+ZY,+ZY         0  
+ZZ,(-1, -1),+ZZ,+Z_         0  
+ZZ,(-1, -1),+ZZ,+_Z         0  
+ZZ,(-1, -1),+ZZ,+ZZ         0  

[972 rows x 450 columns]

In [101]:
df.dtypes

H,+X_        string
H,+Y_        string
H,+Z_        string
H,+_X        string
H,+_Y        string
              ...  
A,+ZZ,+YX    string
A,+ZZ,+YY    string
A,+ZZ,+YZ    string
A,+ZZ,+ZX    string
A,+ZZ,+ZY    string
Length: 450, dtype: object

In [102]:
len(df.index)

972

In [103]:
len(df.columns)

450